In [1]:
!git clone https://github.com/NVlabs/edm2.git
%cd edm2

!pip install click tqdm psutil scipy pillow pandas --quiet

# Return to working directory
%cd /kaggle/working

Cloning into 'edm2'...


remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (17/17), done.


remote: Total 60 (delta 13), reused 10 (delta 10), pack-reused 33 (from 1)


Receiving objects: 100% (60/60), 1.27 MiB | 11.23 MiB/s, done.
Resolving deltas: 100% (24/24), done.


/kaggle/working/edm2


/kaggle/working


In [2]:
import re
from pathlib import Path

dataset_py = Path("/kaggle/working/edm2/training/dataset.py")
text = dataset_py.read_text()

# =======================================================================
# New dataset class — 10-dim conditioning from 5 face landmarks
# =======================================================================

injection = r'''
# ======================================================================
# CelebALandmarkDataset — 5 landmarks (10 coords) conditioning dataset
# ======================================================================

class CelebALandmarkDataset(Dataset):
    def __init__(
        self,
        path,               # Directory containing CelebA 64×64 images
        landmark_csv,       # Path to celeba64_landmarks_transformed.csv
        resolution=None,
        **super_kwargs,
    ):
        import pandas as pd
        import numpy as np
        import os
        import PIL.Image

        self._path = path

        # ------------------------------------------------------------
        # Load landmark CSV
        # ------------------------------------------------------------
        df = pd.read_csv(landmark_csv)

        img_col = df.columns[0]          # "image_id"
        landmark_cols = df.columns[1:]   # 10 columns (x,y for 5 landmarks)

        # Convert to float32
        landmarks = df[landmark_cols].values.astype(np.float32)

        # filename -> landmark vector mapping
        self._landmark_dict = {
            fn: landmarks[i]
            for i, fn in enumerate(df[img_col].values)
        }

        # ------------------------------------------------------------
        # Collect matching images from the folder
        # ------------------------------------------------------------
        all_files = sorted([
            f for f in os.listdir(path)
            if f.lower().endswith(('.jpg', '.jpeg', '.png'))
        ])

        self._image_fnames = [f for f in all_files if f in self._landmark_dict]

        if len(self._image_fnames) == 0:
            raise IOError("No matching images found for landmark dataset.")

        # ------------------------------------------------------------
        # Determine CHW shape from first image
        # ------------------------------------------------------------
        example = np.array(PIL.Image.open(os.path.join(path, self._image_fnames[0])))
        if example.ndim == 2:
            example = np.stack([example] * 3, axis=-1)
        CHW = example.transpose(2,0,1).shape

        raw_shape = [len(self._image_fnames)] + list(CHW)

        super().__init__(
            name="celeba-landmarks",
            raw_shape=raw_shape,
            **super_kwargs
        )

    # ------------------------------------------------------------
    # Load raw image → CHW uint8
    # ------------------------------------------------------------
    def _load_raw_image(self, raw_idx):
        import numpy as np
        import os
        import PIL.Image

        fname = self._image_fnames[raw_idx]
        img = PIL.Image.open(os.path.join(self._path, fname)).convert('RGB')
        img = np.array(img, dtype=np.uint8)
        img = img.transpose(2,0,1)
        return img

    # ------------------------------------------------------------
    # Load raw labels → (10,)
    # ------------------------------------------------------------
    def _load_raw_labels(self):
        import numpy as np
        labels = np.zeros((len(self._image_fnames), 10), dtype=np.float32)
        for i, fname in enumerate(self._image_fnames):
            labels[i] = self._landmark_dict[fname]
        return labels
'''

# Insert before ImageFolderDataset
updated_text = re.sub(
    r"(class ImageFolderDataset\(Dataset\):)",
    injection + r"\n\n\1",
    text,
    count=1,
)

dataset_py.write_text(updated_text)
print("✅ CelebALandmarkDataset successfully added to dataset.py")


✅ CelebALandmarkDataset successfully added to dataset.py


In [3]:
train_py = Path("/kaggle/working/edm2/train_edm2.py")
text = train_py.read_text()

# ------------------------------------------------------------
# Add CLI option
# ------------------------------------------------------------

text = re.sub(
    r"(# Main options\.)",
    r"\1\n@click.option('--landmarks-csv', help='CelebA landmark CSV', type=str, default=None)",
    text,
)

# ------------------------------------------------------------
# Patch setup_training_config()
# ------------------------------------------------------------
import re as _re

setup_pattern = r"def setup_training_config[\s\S]*?return c"
setup_block = _re.search(setup_pattern, text).group(0)

patched_block = setup_block.replace(
    "c.dataset_kwargs = dnnlib.EasyDict(class_name='training.dataset.ImageFolderDataset', path=opts.data, use_labels=opts.get('cond', True))",
    """
    # =========================
    # Select dataset class
    # =========================
    if opts.get('landmarks_csv', None) is not None:
        c.dataset_kwargs = dnnlib.EasyDict(
            class_name='training.dataset.CelebALandmarkDataset',
            path=opts.data,
            landmark_csv=opts.landmarks_csv,
            use_labels=True
        )
    else:
        c.dataset_kwargs = dnnlib.EasyDict(
            class_name='training.dataset.ImageFolderDataset',
            path=opts.data,
            use_labels=opts.get('cond', True)
        )
    """
)

text = text.replace(setup_block, patched_block)
train_py.write_text(text)

print("✅ train_edm2.py patched with --landmarks-csv support.")


✅ train_edm2.py patched with --landmarks-csv support.


In [4]:
import sys
sys.path.append("/kaggle/working/edm2")

import training.dataset
import training.encoders
import training.networks_edm2

print("CelebALandmarkDataset exists? →", hasattr(training.dataset, "CelebALandmarkDataset"))


CelebALandmarkDataset exists? → True


In [5]:
!torchrun --standalone --nproc_per_node=1 /kaggle/working/edm2/train_edm2.py \
    --outdir=/kaggle/working/edm2-landmark-cond \
    --data=/kaggle/input/celeva-64x64-dataset/celeba64/train \
    --landmarks-csv=/kaggle/input/bbox-and-landmark-conditioning-csv-aligned-files/celeba64_landmarks_transformed.csv \
    --cond=True \
    --preset=edm2-img64-xs \
    --batch=64 \
    --batch-gpu=32 \
    --duration=1572Ki \
    --status=16Ki \
    --snapshot=128Ki \
    --checkpoint=0 \
    --seed=0


[W1209 04:43:08.684631980 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W1209 04:43:08.685393058 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3


[W1209 04:43:10.530670992 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W1209 04:43:10.531428909 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3


Setting up training config...



Training config:
{
  "dataset_kwargs": {
    "class_name": "training.dataset.CelebALandmarkDataset",
    "path": "/kaggle/input/celeva-64x64-dataset/celeba64/train",
    "landmark_csv": "/kaggle/input/bbox-and-landmark-conditioning-csv-aligned-files/celeba64_landmarks_transformed.csv",
    "use_labels": true
  },
  "encoder_kwargs": {
    "class_name": "training.encoders.StandardRGBEncoder"
  },
  "total_nimg": 1609728,
  "batch_size": 64,
  "network_kwargs": {
    "class_name": "training.networks_edm2.Precond",
    "model_channels": 128,
    "dropout": 0.0,
    "use_fp16": true
  },
  "loss_kwargs": {
    "class_name": "training.training_loop.EDM2Loss",
    "P_mean": -0.8,
    "P_std": 1.6
  },
  "lr_kwargs": {
    "func_name": "training.training_loop.learning_rate_schedule",
    "ref_lr": 0.012,
    "ref_batches": 35000
  },
  "batch_gpu": 32,
  "loss_scaling": 1.0,
  "cudnn_benchmark": true,
  "status_nimg": 16384,
  "snapshot_nimg": 131072,
  "checkpoint_nimg": null,
  "seed": 0
}

[rank0]:[W1209 04:43:16.689438917 ProcessGroupNCCL.cpp:4561] [PG ID 0 PG GUID 0 Rank 0]  using GPU 0 to perform barrier as devices used by this process are currently unknown. This can potentially cause a hang if this rank to GPU mapping is incorrect. Specify device_ids in barrier() to force use of a particular device, or call init_process_group() with a device_id.


...

done


Saving network-snapshot-0001572-0.100.pkl ... 

done


Status: kimg 1589.2    time 11h 51m 26s  sec/tick 440.49   sec/kimg 26.817  maintenance 1.12    cpumem 3.53   gpumem 8.19   reserved 8.58  


Status: kimg 1605.6    time 11h 58m 44s  sec/tick 438.64   sec/kimg 26.768  maintenance 0.08    cpumem 3.53   gpumem 8.19   reserved 8.58  
